<a href="https://colab.research.google.com/github/awagler2/NODE/blob/main/scorecard_merge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

# Load the datasets
df_imputed = pd.read_csv('full_imputed_final.csv')
df_field_of_study = pd.read_csv('FieldOfStudyData1920_2021_PP.csv')
hrsu=pd.read_csv('HRSU.csv')
# Merge the dataframes on the 'UNITID' column
merged_df = pd.merge(df_imputed, df_field_of_study, left_on='unitid',right_on='UNITID')

# Display the first few rows of the merged dataframe to verify
print(merged_df.shape)
print(df_imputed.shape)



<ipython-input-1-c52e111077ca>:4: DtypeWarning: Columns (26,29,30,501,502) have mixed types. Specify dtype option on import or set low_memory=False.
  df_imputed = pd.read_csv('full_imputed_final.csv')


(1311135, 684)
(23441, 510)


In [5]:
from IPython import get_ipython
from IPython.display import display
# %%
import pandas as pd

# Load the datasets
#df = pd.read_csv('ScoreCard_IPEDS.csv')
# Load the NSF_CIP_codes.csv and specify 'CIP Code' as string type
nsf=pd.read_csv('NSF_CIP_codes.csv', dtype={'CIP Code': str})

#nsf.head()
# %%
#print(df['CIPCODE'].head())
nsf['CIP2']=nsf['CIP Code']
# Now that 'CIP2' is guaranteed to be string type, .str accessor will work
nsf['CIP2'] = nsf['CIP2'].str.replace('.', '', regex=False)
merged_df['STEM_CIP'] = merged_df['CIPCODE'].apply(lambda x: x if str(x) in nsf['CIP Code'].values else 'Non-STEM') # Convert x to string for consistent comparison
merged_df['STEM'] = merged_df['CIPCODE'].apply(lambda x: 1 if str(x) in nsf['CIP2'].values else 0) # Convert x to string for consistent comparison

print(merged_df['STEM'].value_counts())
print(merged_df.shape)
#merged_df.to_csv('ScoreCard_IPEDS_CIP.csv', index=False)

#merged_df.to_csv('ScoreCard_IPEDS.csv', index=False)

STEM
0    1047556
1     263579
Name: count, dtype: int64
(1311135, 687)


In [7]:
import pandas as pd

hrsu=pd.read_csv('HRSU.csv')
#df=pd.read_csv('ScoreCard_IPEDS_CIP.csv')
#change to character strings
#hrsu['UNITID']=hrsu['UNITID'].astype(str)
#df['UNITID']=df['UNITID'].astype(str)
#check these are same type for comparison
print(hrsu['UNITID'].dtype)
print(merged_df['UNITID'].dtype)

# Create a binary variable 'HRSU' in d#f
merged_df['HRSU'] = merged_df['UNITID'].apply(lambda x: 1 if x in hrsu['UNITID'].values else 0)

# Print the value counts to check the result
print(merged_df['HRSU'].value_counts())
print(hrsu['UNITID'].values)

merged_df.to_csv('ScoreCard_IPEDS_CIP_HRSU.csv', index=False)


int64
float64
HRSU
0    1216095
1      95040
Name: count, dtype: int64
[104151 190576 133951 185828 228723 229115 228769 228778 228796 229027
 104179 110653 110671 110705 110714 132903 126562 225511 145600 182281
 187985 227216 122409 133669 188030 445188 110644]
